In [ ]:
import mxnet as mx
import numpy as np
import cv2,sys,time
from IPython.display import Image
from collections import namedtuple
import os

In [ ]:
os.system("cp -f ../models/synset.txt .")
os.system("cp -f ../models/Inception-BN*.* .")

In [ ]:
def loadModel(modelname):
    sym, arg_params, aux_params = mx.model.load_checkpoint(modelname, 0)
    arg_params['prob_label'] = mx.nd.array([0])
    mod = mx.mod.Module(symbol=sym)
    mod.bind(for_training=False, data_shapes=[('data', (1,3,224,224))])
    mod.set_params(arg_params, aux_params)
    return mod

In [ ]:
def loadCategories():
    synsetfile = open('synset.txt', 'r')
    synsets = []
    for l in synsetfile:
        synsets.append(l.rstrip())
    return synsets

In [ ]:
def prepareNDArray(filename):
    img = cv2.imread(filename)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224,))
    img = np.swapaxes(img, 0, 2)
    img = np.swapaxes(img, 1, 2)
    img = img[np.newaxis, :]
    return mx.nd.array(img) 

In [ ]:
def predict(filename, model, categories, n):
    array = prepareNDArray(filename)
    Batch = namedtuple('Batch', ['data'])
    t1 = time.time()
    model.forward(Batch([array]))
    prob = model.get_outputs()[0].asnumpy()
    t2 = time.time()
    print (t2-t1)
    prob = np.squeeze(prob)
    sortedprobindex = np.argsort(prob)[::-1]
    topn = []
    for i in sortedprobindex[0:n]:
        topn.append((prob[i], categories[i]))
    return topn

In [ ]:
def init(modelname): 
    model = loadModel(modelname)
    cats = loadCategories()
    return model, cats

In [ ]:
os.system("wget http://ghk.h-cdn.co/assets/17/18/480x720/gallery-gettyimages-462376265.jpg")
filename = "gallery-gettyimages-462376265.jpg"
Image(filename=filename)

In [ ]:
inceptionv3,c = init("Inception-BN")

In [ ]:
print predict(filename,inceptionv3,c,5)